In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from geopy.distance import geodesic

from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [130]:
from scipy import interpolate
import numpy as np

In [3]:
shike = pd.read_excel('C:\intern\data\docs_20220525\时刻表\shike.xlsx')
shike_sorted = shike.sort_values(by=['路线', '车次id', '站序'])
checi = pd.read_excel('C:/intern/data/docs_20220525/时刻表/checi.xlsx')
gps = pd.read_excel('C:/intern/data/docs_20220525/GPS/gps_info.xlsx')
bus = pd.read_excel('C:/intern/data/docs_20220525/GPS/bus_info.xlsx')
duiying = pd.read_excel('C:/intern/data/docs_20220525/时刻表/duiying.xlsx')
gps_info_up = pd.read_excel('C:/intern/data/docs_20220525/GPS/April_up.xlsx')
site_gps = pd.read_excel('C:/intern/data/docs_20220525/路线站点信息/site_gps.xlsx')

In [4]:
gps = gps[(gps.bus_id!='\\N') & (gps.bus_id!='-1') & (gps.bus_id!='0')]

In [68]:
site_gps.drop ('Y路线', axis=1, inplace=True)
site_gps.drop_duplicates(inplace = True)

In [5]:
#新增‘接收日期’column
gps_info_up['接收日期'] = gps_info_up['接收时间'].str[0:10]
gps_info_up['接收时间']= pd.to_datetime (gps_info_up ['接收时间'])

In [6]:
# 导入GPS有问题的车辆数据
bus_wrong_gps_df = pd.read_excel('C:/intern/data/docs_updated/GPS/GPS数据问题.xlsx', sheet_name='gps数据间隔超过1分钟车辆')
# 去除异常值
gps_info_up_cleaned = gps_info_up[~gps_info_up['gps串号'].isin(bus_wrong_gps_df['gps_sn'].values)]

In [21]:
#计算位置和站点的距离
def distance(curr_long, curr_lat, station_long, station_lat):
    curr_ = (curr_lat, curr_long)
    station_ = (station_lat, station_long)
 
    return (geodesic(curr_, station_).km)
    #return ((curr_long-station_long)**2+(curr_lat-station_lat)**2)**(1/2)
    #return (abs(curr_long-station_long)+abs(curr_lat-station_lat))

In [8]:
#find nearest station
def possible_nearest_station(site_df, shike_df, this_long, this_lat, time, date):
    time_format = "%Y-%m-%d %H:%M:%S"
    min_dist = float('inf')
    res = 'Not Found'

    for index, row in site_df.iterrows():
        site_long = row['经度']
        site_lat = row['纬度']
        dist = distance(this_long, this_lat, site_long, site_lat)
        if dist < min_dist:          
            min_dist = dist
            res = row['站点'] 

    return res, min_dist

In [163]:
def total_seconds(start, end):
    
    return end-start+1

In [218]:
#对gps数据进行插值,返回插值过的gps_df
def scipy_interpolation(gps_df):
    original_times = gps_df['接收时间'].values
    original_jingdu = gps_df['经度'].values
    original_weidu = gps_df['纬度'].values
    start_time = original_times[0]
    end_time = original_times[len(gps_df['接收时间'])-1]
    time_base = start_time
    xnew_datetime = pd.date_range(start_time, end_time, freq='S')
    
    if len(gps_df)>1:
        # Initialize input values x and y

        x = list(map(lambda a: pd.Timedelta(a-time_base).total_seconds(), original_times))
        y_jingdu = original_jingdu
        y_weidu = original_weidu
        total_row_num = total_seconds(x[0],x[len(x)-1])

        # Interpolation
        intpl_jingdu = interpolate.interp1d(x, y_jingdu)
        intpl_weidu = interpolate.interp1d(x, y_weidu)

        xnew = np.arange(0, total_row_num, 1)
        ynew_jingdu = intpl_jingdu(xnew)
        ynew_weidu = intpl_weidu(xnew)

        gps_sn = gps_df['gps串号'].values[0]
        info_date = gps_df['接收日期'].values[0]
        new_df = pd.DataFrame({'接收时间': xnew_datetime, '经度': ynew_jingdu, '纬度': ynew_weidu})
        new_df['gps串号'] = gps_sn
        new_df['接收日期'] = info_date
        
        return new_df
    else:
        
        return gps_df
    
    


In [9]:
#对gps数据进行插值,返回插值过的gps_df
def linear_interpolation(gps_df):
    new_len = 2 * len(gps_df) -1
    len_ = len(gps_df)
    new_df = pd.DataFrame(columns=['序号', 'gps串号', 'gps状态', '经度', '纬度', '接收时间', '速度', '方向','电源状态', '接收日期'])
    
    for i in range(new_len):
        #even line is original data
        if(i % 2 == 0):
            last_row = gps_df.iloc[i//2]
            if (i//2 + 1) < len_:
                next_row = gps_df.iloc[(i//2 + 1)]
            
            new_df.loc[len(new_df.index)] = [last_row['序号'], last_row['gps串号'], last_row['gps状态'], last_row['经度'], last_row['纬度'], last_row['接收时间'], last_row['速度'], last_row['方向'], last_row['电源状态'], last_row['接收日期']]
        
        #odd line is filled data
        else:
            mid_jingdu = (last_row['经度']+next_row['经度'])/2
            mid_weidu = (last_row['纬度']+next_row['纬度'])/2
            new_time = last_row['接收时间'] + timedelta(0,10) 
            new_df.loc[len(new_df.index)] = [0, last_row['gps串号'], last_row['gps状态'], mid_jingdu, mid_weidu, new_time, last_row['速度'], last_row['方向'], last_row['电源状态'], last_row['接收日期']]
    
    return new_df
    
    

In [213]:
def find_stations(dates, data, site_gps, time_start, time_end, site_start, site_end, interpolation):
    stop_df = pd.DataFrame(columns=['日期','站点','距离'])
    pass_df = pd.DataFrame(columns=['日期','站点','距离'])
    
    site1 = site_gps[site_gps['站点'].str.contains(site_start)]
    site2 = site_gps[site_gps['站点'].str.contains(site_end)]
    #if len(site2)==0: print(site_end)
    start_jingdu = site1['经度'].values[0]
    start_weidu = site1['纬度'].values[0]
    end_jingdu = site2['经度'].values[0]
    end_weidu = site2['纬度'].values[0]
    stay_limit = 9
    stay_standard = 2
    if interpolation: 
        stay_limit = 180
        stay_standard = 30
    
    for eachD_id in range(len(dates)):  
        each_stop_df = pd.DataFrame(columns=['日期','站点','距离'])
        each_pass_df = pd.DataFrame(columns=['日期','站点','距离'])
        #data at this date
        print(dates[eachD_id])
        try_data = data[data['接收日期']==dates[eachD_id]]   
        if interpolation:
            try_data = scipy_interpolation(try_data)
        
        print(len(try_data))
        
        #position info
        last_longitude = 0
        last_latitude = 0 
        count_stay = 0  
        
        last_nearest = ''
            
        time_lower, time_upper = time_lower_upper(time_start, time_end, dates[eachD_id])
        print(time_lower, time_upper)
        start_check = 0
        end_check = 0
        
        
        #iterate signals
        for index1, row1 in try_data.iterrows():
            
            if (row1['接收时间']>time_lower) & (row1['接收时间']<time_upper):
                #find nearest station
                nearest_, distance_ = possible_nearest_station(site_gps, shike, last_longitude, last_latitude, row1['接收时间'],dates[eachD_id])
                
                if ~start_check:
                    if distance(row1['经度'], row1['纬度'], start_jingdu, start_weidu) <= 0.5:
                        start_check = 1
                elif ~end_check:
                    if distance(row1['经度'], row1['纬度'], end_jingdu, end_weidu) <= 0.5:
                        end_check = 1
                if (~end_check) & (start_check) & (nearest_ != last_nearest) & (distance_<=1):
                    each_pass_df.loc[len(each_pass_df.index)] = [dates[eachD_id], nearest_, distance_]
                    last_nearest = nearest_    
                        
                """
                if (nearest_ != last_nearest) & (distance<=1):
                    pass_df.loc[len(pass_df.index)] = [dates[eachD_id], nearest_, distance]
                    last_nearest = nearest_    
                """
                        
               
                #position unchanged 
                if (row1['经度']==last_longitude) | (row1['纬度']==last_latitude):
                    count_stay += 1

                    #check duration
                    if count_stay == stay_standard:
                        if (distance_ <=1) & (~end_check) & (start_check):
                            each_stop_df.loc[len(each_stop_df.index)] = [dates[eachD_id], nearest_, distance_]
                        else:
                            count_stay = 0 

                    elif count_stay>stay_limit:
                        each_stop_df.drop(each_stop_df.tail(1).index,inplace=True)
                        count_stay = 0         
                else:
                    count_stay = 0
                    last_longitude = row1['经度']
                    last_latitude = row1['纬度']
                    
        each_stop_df = each_stop_df.drop_duplicates(subset='站点', keep='first')
        each_pass_df = each_pass_df.drop_duplicates(subset='站点', keep='first')
        
        stop_df = stop_df.append(each_stop_df)
        pass_df = pass_df.append(each_pass_df)
                    
            
        #stop_df = stop_df.drop(stop_df[stop_df['距离'] > 1].index)
        #pass_df = pass_df.drop(pass_df[pass_df['距离'] > 1].index)
                
    return stop_df, pass_df

In [11]:
def run_sn(this_sn):
    this_data = gps_info_up_cleaned[gps_info_up_cleaned['gps串号']==int(this_sn)]
    this_data = this_data.sort_values(by='接收时间')

    this_bus_id = int(gps[gps['gps_sn']==this_sn].bus_id)
    this_bus_plate = bus[bus['id']==this_bus_id].bus_plate.values[0] 

    this_signal_dates = this_data['接收日期'].unique()

    """
    #valid start stations
    this_valid_stations = this_zhandian_linked[this_zhandian_linked['站序']==1]
    this_station_start = this_valid_stations['站点'].values
    this_checi_start = this_valid_stations['车次id'].values
    this_time_start_true = this_valid_stations['时间'].values

    #valid end stations
    this_last_valid = this_zhandian_linked.groupby('车次id').last()
    this_last_valid = this_last_valid.reset_index()
    this_station_end = this_last_valid['站点'].values
    this_checi_end = this_last_valid['车次id'].values
    this_time_end_true = this_last_valid['时间'].values
    """
    
    stop_df, pass_df = find_stations(this_signal_dates, this_data, site_gps)
    
    
    return stop_df, pass_df

In [206]:
def time_lower_upper(time_start, time_end, date):
    time_format = "%Y-%m-%d %H:%M:%S"
    time_supposed_start = date+' '+time_start
    time_supposed_end = date+' '+time_end
    time_supposed_start = datetime.strptime(time_supposed_start, time_format)
    time_supposed_end = datetime.strptime(time_supposed_end, time_format)
    #time_interval = (time_supposed_end - time_supposed_start).total_seconds()
    #time_lower = time_supposed_start - timedelta(0, time_interval/2)
    #time_upper = time_supposed_end + timedelta(0, time_interval/2)
    time_lower = time_supposed_start - timedelta(minutes = 5)
    time_upper = time_supposed_end + timedelta(minutes = 5)
    
    
    return time_lower, time_upper

In [104]:
def run_checi(this_checi, interpolation):
    this_bus_plate = duiying[duiying['车次id']==this_checi]['车牌号'].values[0]
    this_bus_id = []
    this_sn = []
    empty_df = pd.DataFrame(columns=['日期','站点','距离'])
    
    this_bus_id_list = bus[bus['bus_plate']==this_bus_plate]['id'].values
    if len(this_bus_id_list)>0:
        this_bus_id = this_bus_id_list[0]
    else:
        return empty_df, empty_df
    
    this_sn_list = gps[gps['bus_id']==str(this_bus_id)]['gps_sn'].values

    
    if len(this_sn_list) >0:
        this_sn = this_sn_list[0]
    else:
        return empty_df, empty_df
        
    """   
    this_bus_id = bus[bus['bus_plate']==this_bus_plate]['id'].values[0]
    this_sn = gps[gps['bus_id']==str(this_bus_id)]['gps_sn'].values[0]
    """ 
    
    this_data = gps_info_up_cleaned[gps_info_up_cleaned['gps串号']==int(this_sn)]
    this_data = this_data.sort_values(by='接收时间')

    this_signal_dates = this_data['接收日期'].unique()

    """
    #valid start stations
    this_valid_stations = this_zhandian_linked[this_zhandian_linked['站序']==1]
    this_station_start = this_valid_stations['站点'].values
    this_checi_start = this_valid_stations['车次id'].values
    this_time_start_true = this_valid_stations['时间'].values

    #valid end stations
    this_last_valid = this_zhandian_linked.groupby('车次id').last()
    this_last_valid = this_last_valid.reset_index()
    this_station_end = this_last_valid['站点'].values
    this_checi_end = this_last_valid['车次id'].values
    this_time_end_true = this_last_valid['时间'].values
    """
    time_start = shike[(shike['车次id']==this_checi) & (shike['站序']==1)]['时间'].values[0]
    chexu_max = shike[shike['车次id']==this_checi]['站序'].max()
    time_end = shike[(shike['车次id']==this_checi) & (shike['站序']==chexu_max)]['时间'].values[0]
    
    site_start = shike[(shike['车次id']==this_checi) & (shike['站序']==1)]['站点'].values[0]
    site_end = shike[(shike['车次id']==this_checi) & (shike['站序']==chexu_max)]['站点'].values[0]
    
    
    stop_df, pass_df = find_stations(this_signal_dates, this_data, site_gps, time_start, time_end, site_start, site_end, interpolation)
    
    
    return stop_df, pass_df

In [90]:
checi['车次id'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

'国电南瑞' 225-243 先排除

In [224]:
#all_checi = [i for i in range(90, 150, 1)]
all_checi = [i for i in range(244, 400, 1)]
all_checi 

[244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 291,
 292,
 293,
 294,
 295,
 296,
 297,
 298,
 299,
 300,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 331,
 332,
 333,
 334,
 335,
 336,
 337,
 338,
 339,
 340,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 396,
 397,
 398,
 399]

In [212]:
pass_with_interpolation[pass_with_interpolation[]]

,日期,站点,距离,车次
0,2022-04-01,成贤地铁站,0.511097,0
1,2022-04-01,星火路,0.482988,0
2,2022-04-01,创智大厦,0.212827,0
3,2022-04-01,南京集成电路大学,0.989414,0
4,2022-04-01,南京工业大学（天浦路西公交站）,0.978256,0
...,...,...,...,...
13,2022-04-14,耀华,0.370332,0
14,2022-04-14,青云大厦,0.459756,0
15,2022-04-14,膜园,0.592173,0
18,2022-04-14,共享空间,0.990521,0


In [226]:
#current_checi = 3
stop_df = pd.DataFrame()
stop_interpolation_df = pd.DataFrame()
pass_df = pd.DataFrame()
pass_interpolation_df = pd.DataFrame()
duibi_ = pd.DataFrame(columns=['车次','日期','before_停靠站点数','after_停靠站点数','before_经过站点数','after_经过站点数'])


for checi_ in all_checi:
    print(checi_)
    stop_no_interpolation, pass_no_interpolation = run_checi(checi_, 0)
    stop_with_interpolation, pass_with_interpolation = run_checi(checi_, 1)
    stop_no_interpolation['车次']=checi_
    pass_no_interpolation['车次']=checi_
    stop_with_interpolation['车次']=checi_
    pass_with_interpolation['车次']=checi_
    
    
    stop_df = stop_df.append(stop_no_interpolation)
    stop_interpolation_df = stop_interpolation_df.append(stop_with_interpolation)
    pass_df = pass_df.append(pass_no_interpolation)
    pass_interpolation_df = pass_interpolation_df.append(pass_with_interpolation)
    
    dates = stop_no_interpolation['日期'].unique()
    
    
    for date_ in dates:
        before_stop = len(stop_no_interpolation[stop_no_interpolation['日期']==date_])
        after_stop = len(stop_with_interpolation[stop_with_interpolation['日期']==date_])
        before_pass = len(pass_no_interpolation[pass_no_interpolation['日期']==date_])
        after_pass = len(pass_with_interpolation[pass_with_interpolation['日期']==date_])
        """
        before_stop = len(stop_no_interpolation[stop_no_interpolation['日期']==date_])
        after_stop = len(stop_with_interpolation[stop_with_interpolation['日期']==date_])
        before_pass = len(pass_no_interpolation[pass_no_interpolation['日期']==date_])
        after_pass = len(pass_with_interpolation[pass_with_interpolation['日期']==date_])
        """
        duibi_.loc[len(duibi_.index)] = [checi_, date_, before_stop, after_stop, before_pass, after_pass]

duibi_.to_excel('C:/intern/data/docs_20220525/按秒插值/插值前后经停靠站点数对比244-399.xlsx',index=False)
stop_df.to_excel('C:/intern/data/docs_20220525/按秒插值/插值前停靠站点244-399.xlsx',index=False)
stop_interpolation_df.to_excel('C:/intern/data/docs_20220525/按秒插值/插值后停靠站点244-399.xlsx',index=False)
pass_df.to_excel('C:/intern/data/docs_20220525/按秒插值/插值前经过站点244-399.xlsx',index=False)
pass_interpolation_df.to_excel('C:/intern/data/docs_20220525/按秒插值/插值后经过站点244-399.xlsx',index=False)

244
2022-04-01
677
2022-04-01 07:45:00 2022-04-01 08:20:00
2022-04-02
693
2022-04-02 07:45:00 2022-04-02 08:20:00
2022-04-03
4
2022-04-03 07:45:00 2022-04-03 08:20:00
2022-04-04
7
2022-04-04 07:45:00 2022-04-04 08:20:00
2022-04-05
2
2022-04-05 07:45:00 2022-04-05 08:20:00
2022-04-06
762
2022-04-06 07:45:00 2022-04-06 08:20:00
2022-04-07
761
2022-04-07 07:45:00 2022-04-07 08:20:00
2022-04-08
687
2022-04-08 07:45:00 2022-04-08 08:20:00
2022-04-09
3
2022-04-09 07:45:00 2022-04-09 08:20:00
2022-04-10
4
2022-04-10 07:45:00 2022-04-10 08:20:00
2022-04-11
703
2022-04-11 07:45:00 2022-04-11 08:20:00
2022-04-12
803
2022-04-12 07:45:00 2022-04-12 08:20:00
2022-04-13
332
2022-04-13 07:45:00 2022-04-13 08:20:00
2022-04-14
642
2022-04-14 07:45:00 2022-04-14 08:20:00
2022-04-01
67936
2022-04-01 07:45:00 2022-04-01 08:20:00
2022-04-02
55667
2022-04-02 07:45:00 2022-04-02 08:20:00
2022-04-03
47151
2022-04-03 07:45:00 2022-04-03 08:20:00
2022-04-04
69520
2022-04-04 07:45:00 2022-04-04 08:20:00
2022-04-

2022-04-07
791
2022-04-07 08:30:00 2022-04-07 09:05:00
2022-04-08
760
2022-04-08 08:30:00 2022-04-08 09:05:00
2022-04-09
197
2022-04-09 08:30:00 2022-04-09 09:05:00
2022-04-10
148
2022-04-10 08:30:00 2022-04-10 09:05:00
2022-04-11
944
2022-04-11 08:30:00 2022-04-11 09:05:00
2022-04-12
894
2022-04-12 08:30:00 2022-04-12 09:05:00
2022-04-13
407
2022-04-13 08:30:00 2022-04-13 09:05:00
2022-04-14
805
2022-04-14 08:30:00 2022-04-14 09:05:00
2022-04-01
53892
2022-04-01 08:30:00 2022-04-01 09:05:00
2022-04-02
50560
2022-04-02 08:30:00 2022-04-02 09:05:00
2022-04-03
40230
2022-04-03 08:30:00 2022-04-03 09:05:00
2022-04-04
35524
2022-04-04 08:30:00 2022-04-04 09:05:00
2022-04-05
14914
2022-04-05 08:30:00 2022-04-05 09:05:00
2022-04-06
67207
2022-04-06 08:30:00 2022-04-06 09:05:00
2022-04-07
67988
2022-04-07 08:30:00 2022-04-07 09:05:00
2022-04-08
50851
2022-04-08 08:30:00 2022-04-08 09:05:00
2022-04-09
35520
2022-04-09 08:30:00 2022-04-09 09:05:00
2022-04-10
19104
2022-04-10 08:30:00 2022-04-10

2022-04-13
1354
2022-04-13 09:05:00 2022-04-13 09:40:00
2022-04-14
2716
2022-04-14 09:05:00 2022-04-14 09:40:00
2022-04-01
53977
2022-04-01 09:05:00 2022-04-01 09:40:00
2022-04-02
50653
2022-04-02 09:05:00 2022-04-02 09:40:00
2022-04-03
50857
2022-04-03 09:05:00 2022-04-03 09:40:00
2022-04-04
51058
2022-04-04 09:05:00 2022-04-04 09:40:00
2022-04-05
53961
2022-04-05 09:05:00 2022-04-05 09:40:00
2022-04-06
51107
2022-04-06 09:05:00 2022-04-06 09:40:00
2022-04-07
52021
2022-04-07 09:05:00 2022-04-07 09:40:00
2022-04-08
50957
2022-04-08 09:05:00 2022-04-08 09:40:00
2022-04-09
50938
2022-04-09 09:05:00 2022-04-09 09:40:00
2022-04-10
50818
2022-04-10 09:05:00 2022-04-10 09:40:00
2022-04-11
51122
2022-04-11 09:05:00 2022-04-11 09:40:00
2022-04-12
51068
2022-04-12 09:05:00 2022-04-12 09:40:00
2022-04-13
26073
2022-04-13 09:05:00 2022-04-13 09:40:00
2022-04-14
50659
2022-04-14 09:05:00 2022-04-14 09:40:00
261
262
263
2022-04-01
957
2022-04-01 09:20:00 2022-04-01 09:55:00
2022-04-02
888
2022-04-

2022-04-05
17631
2022-04-05 17:25:00 2022-04-05 18:00:00
2022-04-06
66142
2022-04-06 17:25:00 2022-04-06 18:00:00
2022-04-07
50184
2022-04-07 17:25:00 2022-04-07 18:00:00
2022-04-08
56085
2022-04-08 17:25:00 2022-04-08 18:00:00
2022-04-09
41955
2022-04-09 17:25:00 2022-04-09 18:00:00
2022-04-10
67031
2022-04-10 17:25:00 2022-04-10 18:00:00
2022-04-11
49353
2022-04-11 17:25:00 2022-04-11 18:00:00
2022-04-12
50968
2022-04-12 17:25:00 2022-04-12 18:00:00
2022-04-13
37758
2022-04-13 17:25:00 2022-04-13 18:00:00
2022-04-14
71517
2022-04-14 17:25:00 2022-04-14 18:00:00
270
2022-04-01
2547
2022-04-01 17:30:00 2022-04-01 18:05:00
2022-04-02
2726
2022-04-02 17:30:00 2022-04-02 18:05:00
2022-04-03
2502
2022-04-03 17:30:00 2022-04-03 18:05:00
2022-04-04
2515
2022-04-04 17:30:00 2022-04-04 18:05:00
2022-04-05
2545
2022-04-05 17:30:00 2022-04-05 18:05:00
2022-04-06
2791
2022-04-06 17:30:00 2022-04-06 18:05:00
2022-04-07
2793
2022-04-07 17:30:00 2022-04-07 18:05:00
2022-04-08
2736
2022-04-08 17:30:0

2022-04-11
51724
2022-04-11 18:10:00 2022-04-11 18:45:00
2022-04-12
49656
2022-04-12 18:10:00 2022-04-12 18:45:00
2022-04-13
19176
2022-04-13 18:10:00 2022-04-13 18:45:00
2022-04-14
49583
2022-04-14 18:10:00 2022-04-14 18:45:00
279
2022-04-01
677
2022-04-01 18:15:00 2022-04-01 18:50:00
2022-04-02
693
2022-04-02 18:15:00 2022-04-02 18:50:00
2022-04-03
4
2022-04-03 18:15:00 2022-04-03 18:50:00
2022-04-04
7
2022-04-04 18:15:00 2022-04-04 18:50:00
2022-04-05
2
2022-04-05 18:15:00 2022-04-05 18:50:00
2022-04-06
762
2022-04-06 18:15:00 2022-04-06 18:50:00
2022-04-07
761
2022-04-07 18:15:00 2022-04-07 18:50:00
2022-04-08
687
2022-04-08 18:15:00 2022-04-08 18:50:00
2022-04-09
3
2022-04-09 18:15:00 2022-04-09 18:50:00
2022-04-10
4
2022-04-10 18:15:00 2022-04-10 18:50:00
2022-04-11
703
2022-04-11 18:15:00 2022-04-11 18:50:00
2022-04-12
803
2022-04-12 18:15:00 2022-04-12 18:50:00
2022-04-13
332
2022-04-13 18:15:00 2022-04-13 18:50:00
2022-04-14
642
2022-04-14 18:15:00 2022-04-14 18:50:00
2022-04-

...

2022-04-04
51066
2022-04-04 17:05:00 2022-04-04 17:22:00
2022-04-05
53944
2022-04-05 17:05:00 2022-04-05 17:22:00
2022-04-06
51113
2022-04-06 17:05:00 2022-04-06 17:22:00
2022-04-07
52027
2022-04-07 17:05:00 2022-04-07 17:22:00
2022-04-08
50951
2022-04-08 17:05:00 2022-04-08 17:22:00
2022-04-09
50943
2022-04-09 17:05:00 2022-04-09 17:22:00
2022-04-10
50800
2022-04-10 17:05:00 2022-04-10 17:22:00
2022-04-11
51115
2022-04-11 17:05:00 2022-04-11 17:22:00
2022-04-12
51060
2022-04-12 17:05:00 2022-04-12 17:22:00
2022-04-13
26070
2022-04-13 17:05:00 2022-04-13 17:22:00
2022-04-14
50674
2022-04-14 17:05:00 2022-04-14 17:22:00
375
2022-04-01
1739
2022-04-01 17:10:00 2022-04-01 17:32:00
2022-04-02
1633
2022-04-02 17:10:00 2022-04-02 17:32:00
2022-04-03
1
2022-04-03 17:10:00 2022-04-03 17:32:00
2022-04-06
1739
2022-04-06 17:10:00 2022-04-06 17:32:00
2022-04-07
1901
2022-04-07 17:10:00 2022-04-07 17:32:00
2022-04-08
1860
2022-04-08 17:10:00 2022-04-08 17:32:00
2022-04-09
1818
2022-04-09 17:10:00 

2022-04-10
1681
2022-04-10 18:15:00 2022-04-10 18:32:00
2022-04-11
1836
2022-04-11 18:15:00 2022-04-11 18:32:00
2022-04-12
1880
2022-04-12 18:15:00 2022-04-12 18:32:00
2022-04-13
940
2022-04-13 18:15:00 2022-04-13 18:32:00
2022-04-14
1818
2022-04-14 18:15:00 2022-04-14 18:32:00
2022-04-01
53976
2022-04-01 18:15:00 2022-04-01 18:32:00
2022-04-02
50579
2022-04-02 18:15:00 2022-04-02 18:32:00
2022-04-03
1
2022-04-03 18:15:00 2022-04-03 18:32:00
2022-04-06
48541
2022-04-06 18:15:00 2022-04-06 18:32:00
2022-04-07
52025
2022-04-07 18:15:00 2022-04-07 18:32:00
2022-04-08
50975
2022-04-08 18:15:00 2022-04-08 18:32:00
2022-04-09
50914
2022-04-09 18:15:00 2022-04-09 18:32:00
2022-04-10
50823
2022-04-10 18:15:00 2022-04-10 18:32:00
2022-04-11
51126
2022-04-11 18:15:00 2022-04-11 18:32:00
2022-04-12
51061
2022-04-12 18:15:00 2022-04-12 18:32:00
2022-04-13
26073
2022-04-13 18:15:00 2022-04-13 18:32:00
2022-04-14
50673
2022-04-14 18:15:00 2022-04-14 18:32:00
381
2022-04-01
1739
2022-04-01 18:35:00 2

2022-04-02
70134
2022-04-02 19:55:00 2022-04-02 20:12:00
2022-04-03
1
2022-04-03 19:55:00 2022-04-03 20:12:00
2022-04-06
50888
2022-04-06 19:55:00 2022-04-06 20:12:00
2022-04-07
66831
2022-04-07 19:55:00 2022-04-07 20:12:00
2022-04-08
79783
2022-04-08 19:55:00 2022-04-08 20:12:00
2022-04-09
48302
2022-04-09 19:55:00 2022-04-09 20:12:00
2022-04-10
1
2022-04-10 19:55:00 2022-04-10 20:12:00
2022-04-11
49835
2022-04-11 19:55:00 2022-04-11 20:12:00
2022-04-12
68777
2022-04-12 19:55:00 2022-04-12 20:12:00
2022-04-13
25951
2022-04-13 19:55:00 2022-04-13 20:12:00
2022-04-14
68775
2022-04-14 19:55:00 2022-04-14 20:12:00
387
2022-04-01
1611
2022-04-01 20:20:00 2022-04-01 20:37:00
2022-04-02
1899
2022-04-02 20:20:00 2022-04-02 20:37:00
2022-04-03
1
2022-04-03 20:20:00 2022-04-03 20:37:00
2022-04-06
1970
2022-04-06 20:20:00 2022-04-06 20:37:00
2022-04-07
1939
2022-04-07 20:20:00 2022-04-07 20:37:00
2022-04-08
1934
2022-04-08 20:20:00 2022-04-08 20:37:00
2022-04-09
1904
2022-04-09 20:20:00 2022-04-

2022-04-06
51113
2022-04-06 17:12:00 2022-04-06 17:29:00
2022-04-07
52027
2022-04-07 17:12:00 2022-04-07 17:29:00
2022-04-08
50951
2022-04-08 17:12:00 2022-04-08 17:29:00
2022-04-09
50943
2022-04-09 17:12:00 2022-04-09 17:29:00
2022-04-10
50800
2022-04-10 17:12:00 2022-04-10 17:29:00
2022-04-11
51115
2022-04-11 17:12:00 2022-04-11 17:29:00
2022-04-12
51060
2022-04-12 17:12:00 2022-04-12 17:29:00
2022-04-13
26070
2022-04-13 17:12:00 2022-04-13 17:29:00
2022-04-14
50674
2022-04-14 17:12:00 2022-04-14 17:29:00
394
2022-04-01
1739
2022-04-01 17:22:00 2022-04-01 17:40:00
2022-04-02
1633
2022-04-02 17:22:00 2022-04-02 17:40:00
2022-04-03
1
2022-04-03 17:22:00 2022-04-03 17:40:00
2022-04-06
1739
2022-04-06 17:22:00 2022-04-06 17:40:00
2022-04-07
1901
2022-04-07 17:22:00 2022-04-07 17:40:00
2022-04-08
1860
2022-04-08 17:22:00 2022-04-08 17:40:00
2022-04-09
1818
2022-04-09 17:22:00 2022-04-09 17:40:00
2022-04-10
1681
2022-04-10 17:22:00 2022-04-10 17:40:00
2022-04-11
1836
2022-04-11 17:22:00 20

2022-04-12
1880
2022-04-12 18:22:00 2022-04-12 18:39:00
2022-04-13
940
2022-04-13 18:22:00 2022-04-13 18:39:00
2022-04-14
1818
2022-04-14 18:22:00 2022-04-14 18:39:00
2022-04-01
53976
2022-04-01 18:22:00 2022-04-01 18:39:00
2022-04-02
50579
2022-04-02 18:22:00 2022-04-02 18:39:00
2022-04-03
1
2022-04-03 18:22:00 2022-04-03 18:39:00
2022-04-06
48541
2022-04-06 18:22:00 2022-04-06 18:39:00
2022-04-07
52025
2022-04-07 18:22:00 2022-04-07 18:39:00
2022-04-08
50975
2022-04-08 18:22:00 2022-04-08 18:39:00
2022-04-09
50914
2022-04-09 18:22:00 2022-04-09 18:39:00
2022-04-10
50823
2022-04-10 18:22:00 2022-04-10 18:39:00
2022-04-11
51126
2022-04-11 18:22:00 2022-04-11 18:39:00
2022-04-12
51061
2022-04-12 18:22:00 2022-04-12 18:39:00
2022-04-13
26073
2022-04-13 18:22:00 2022-04-13 18:39:00
2022-04-14
50673
2022-04-14 18:22:00 2022-04-14 18:39:00


duibi_.to_excel('C:/intern/data/docs_20220525/按秒插值/插值前后经停靠站点数对比100-223.xlsx',index=False)
stop_df.to_excel('C:/intern/data/docs_20220525/按秒插值/插值前停靠站点100-223.xlsx',index=False)
stop_interpolation_df.to_excel('C:/intern/data/docs_20220525/按秒插值/插值后停靠站点100-223.xlsx',index=False)
pass_df.to_excel('C:/intern/data/docs_20220525/按秒插值/插值前经过站点100-223.xlsx',index=False)
pass_interpolation_df.to_excel('C:/intern/data/docs_20220525/按秒插值/插值后经过站点100-223.xlsx',index=False)